In [2]:
import openai
import pandas as pd
import numpy as np
import pickle
from transformers import GPT2TokenizerFast
from typing import List

#openai.api_key = 
openai_api_key='XXXX' ### API KEY
openai_api_version ='2023-03-15-preview' ### Details from Azure
deployment_name='XXXX'
EMBEDDING_MODEL = "text-embedding-ada-002" ### Embedding Model

In [3]:
from langchain.document_loaders import CSVLoader
from langchain.chains import RetrievalQA
from langchain.llms import AzureOpenAI
loader = CSVLoader(file_path=r"C:\Users\Desktop\winequality-red.csv")
docs = loader.load()

In [4]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=800, chunk_overlap=1)
texts = text_splitter.split_documents(docs)
texts

[Document(page_content='fixed acidity: 7.4\nvolatile acidity: 0.7\ncitric acid: 0\nresidual sugar: 1.9\nchlorides: 0.076\nfree sulfur dioxide: 11\ntotal sulfur dioxide: 34\ndensity: 0.9978\npH: 3.51\nsulphates: 0.56\nalcohol: 9.4\nquality: 5', metadata={'source': 'C:\\Users\\GhatkaP\\OneDrive - BASF\\Desktop\\winequality-red.csv', 'row': 0}),
 Document(page_content='fixed acidity: 7.8\nvolatile acidity: 0.88\ncitric acid: 0\nresidual sugar: 2.6\nchlorides: 0.098\nfree sulfur dioxide: 25\ntotal sulfur dioxide: 67\ndensity: 0.9968\npH: 3.2\nsulphates: 0.68\nalcohol: 9.8\nquality: 5', metadata={'source': 'C:\\Users\\GhatkaP\\OneDrive - BASF\\Desktop\\winequality-red.csv', 'row': 1}),
 Document(page_content='fixed acidity: 7.8\nvolatile acidity: 0.76\ncitric acid: 0.04\nresidual sugar: 2.3\nchlorides: 0.092\nfree sulfur dioxide: 15\ntotal sulfur dioxide: 54\ndensity: 0.997\npH: 3.26\nsulphates: 0.65\nalcohol: 9.8\nquality: 5', metadata={'source': 'C:\\Users\\GhatkaP\\OneDrive - BASF\\Deskt

In [5]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import AzureOpenAI

embeddings = OpenAIEmbeddings(
    openai_api_base= f"XXXX",
    openai_api_type='azure',
    deployment='XXXX',
    openai_api_key="XXXX",
    chunk_size=1,
)

In [6]:
doc_search = Chroma.from_documents(texts,embeddings)

In [7]:
import os
os.environ["OPENAI_API_BASE"] = "XXXX"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["OPENAI_API_KEY"] = "XXXX"
os.environ["DEPLOYMENT_NAME"] = "gpt-35-turbo-US"

In [8]:
# Import the necessary classes from the langchain library.
from langchain.prompts import PromptTemplate
from langchain.chat_models import AzureChatOpenAI

# Define a prompt template. This is a format for the text input we'll give to our model.
# It tells the model how to structure its response and what to do in different situations.
template = """I will provide you pieces of [Context] to answer the [Question]. \
If you don't know the answer based on [Context] just say that you don't know, don't try to make up an answer. \
[Context]: {context} \
[Question]: {question} \
Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [9]:
chain = RetrievalQA.from_chain_type(AzureChatOpenAI(openai_api_key="XXXX", deployment_name="gpt-35-turbo-US", model_name="gpt-35-turbo",temperature=0),chain_type="stuff",retriever = doc_search.as_retriever(),return_source_documents=True,chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})

In [12]:
question = "Name the columns in the document?"

# Ask our question to the qa_chain, and store the result.
result = chain({"query": question})

# Print out the result
print(result["result"])

The columns in the document are: fixed acidity, volatile acidity, citric acid, residual sugar, chlorides, free sulfur dioxide, total sulfur dioxide, density, pH, sulphates, alcohol, and quality.
